In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install chromium, its driver, and selenium
!apt update
!apt install chromium-chromedriver
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [ ]:
import time
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pathlib
from datetime import datetime
import pytz
from pytz import timezone
from selenium.webdriver.common.keys import Keys

def crawl_content(url, keywords) :
  driver = webdriver.Chrome('chromedriver',options=options)  
  driver.get(url)

  # Wait for initialize, in seconds
  wait = WebDriverWait(driver, 20)

  #search for keywords
  search = driver.find_element_by_xpath('//*[@id="header"]/div/nav/form/div/input')
  search.send_keys(keywords)
  search.send_keys(Keys.RETURN)  

  time.sleep(8)

  article_list = []
  nextPage = True  
  count = 0
  while nextPage:   
  # while nextPage and count < 3:
    time.sleep(8)
    all_news_link = driver.find_elements_by_css_selector("a.d-flex.article.listing.mb-3.pb-3")

    listOflinks=[]
    for el in all_news_link:
      link = el.get_property('href')
      listOflinks.append(link)

    all_field_category = driver.find_elements_by_css_selector('span.field-category') 
    
    listOfField = []
    for i in all_field_category:
      listOfField.append(i.text)

    time.sleep(8)

    #check the last news article crawled
    csv_file_checking = '/content/drive/Shared drives/NLP Assignment/checking_news.csv'
    file = pathlib.Path(csv_file)
    file2 = pathlib.Path(csv_file_checking)    
    valid = True
    condition = False
    i = 0
    while (i < len(listOflinks)) and valid:   
      driver.get(listOflinks[i])
      time.sleep(8)

      field_category = listOfField[i]

      reload = True
      while reload:
        try:
          time_created = driver.execute_script("return document.querySelector('div.article-meta > div:last-child').innerText.split('-').pop().trim();")
          time.sleep(5)
          time_created = datetime.strptime(time_created, '%B %d, %Y  @ %I:%M%p')
          reload = False
        except:
          reload = True

      timezone = pytz.timezone("Asia/Kuala_Lumpur")
      time_created = time_created.astimezone(timezone)
      time_created = time_created.strftime("%Y-%m-%d %H:%M:%S")

      #title
      title = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#main > div > div > div.col > div:nth-child(1) > div > h1 > span"))).text
      
      #author
      try : 
        author =  wait.until(EC.presence_of_element_located((By.XPATH,"//*[@id='main']/div/div/div[1]/div[1]/div/div/div[1]/div[1]/div/span/a"))).text
      
      except:
        author = " "
    
      #content
      content = wait.until(EC.presence_of_all_elements_located((By.TAG_NAME,"p")))

      all_content = []

      for j in content:
        all_content.append(j.text)
      
      if file2.exists():
        existed_news = pd.read_csv(csv_file_checking)

        record = existed_news[existed_news['Keyword'] == keywords]
        if len(record.index) != 0 :
          existed_date = record['Time Created'].iloc[0]
          existed_title = record['Title'].iloc[0]

          date_time = datetime.strptime(time_created, "%Y-%m-%d %H:%M:%S")
          exist_date = datetime.strptime(existed_date, "%Y-%m-%d %H:%M:%S")

          if (date_time >= exist_date) and (title not in existed_title):
            valid = True
            article_details = {'Keyword': keywords, 'Category': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
            article_list.append(article_details)

            if i == (len(listOflinks)-1):
              condition = True

            else:
              condition = False

          else:
            valid = False 
            condition = False
        
        else:
          article_details = {'Keyword': keywords, 'Category': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
          article_list.append(article_details)    
          condition = True

      else:
        article_details = {'Keyword': keywords, 'Category': field_category, 'Time Created': time_created, 'Title' : title, 'Author' : author, 'Content' : all_content}
        article_list.append(article_details)
        condition = True

      i += 1    

    if condition:
      driver.get(url)      

      #search for keywords
      search = driver.find_element_by_xpath('//*[@id="header"]/div/nav/form/div/input')
      search.send_keys(keywords)
      search.send_keys(Keys.RETURN)  
      time.sleep(8)
          
      count += 1
      a = 0 
      while a < count:
        try:
          #go to next page
          next_button = driver.find_element_by_css_selector("#main > div > div.row > div.col > nav > ul > li.page-item.ml-auto > a")
          next_button.click()
        except:
          break
        
        a += 1

    else:
      nextPage = False

  if article_list != []:
    results = pd.DataFrame(article_list)
    results["Time Created"] = pd.to_datetime(results["Time Created"], format="%Y-%m-%d %H:%M:%S")
    results = results.sort_values(by="Time Created")
  
    if file.exists ():
        results.to_csv(csv_file, mode = 'a', index=False, header = False)
    else:
        results.to_csv(csv_file, index=False)
            
    last_row = pd.DataFrame(results.tail(1))
    if file2.exists ():
      record = existed_news[existed_news['Keyword'] == keywords]
      if len(record) != 0:
        last_record_datetime = last_row['Time Created'].iloc[0]

        existed_news.loc[existed_news["Keyword"] == keywords, 'Time Created'] = last_record_datetime
        existed_news.loc[existed_news["Keyword"] == keywords, 'Title'] = last_row['Title'].iloc[0]
        existed_news.to_csv(csv_file_checking, index=False)

      else:
        last_row.to_csv(csv_file_checking, mode = 'a', index=False, header = False)

    else:
      last_row.to_csv(csv_file_checking, index=False)

csv_file = '/content/drive/Shared drives/NLP Assignment/news.csv' 
crawl_content('https://www.nst.com.my/','Covid-19')

final_results = pd.read_csv('/content/drive/Shared drives/NLP Assignment/news.csv')
final_results["Time Created"] = pd.to_datetime(final_results["Time Created"], format="%Y-%m-%d %H:%M:%S")
final_results = final_results.sort_values(by="Time Created")
final_results = final_results.drop_duplicates(subset ="Title",keep = False)
final_results.to_csv(csv_file, index=False)